In [1]:
import pickle  # pickle 모듈을 import하여 바이너리 형식으로 저장된 데이터를 읽고 쓸 수 있도록 함

with open('./res/rag_data.pkl', 'rb') as f:  # 바이너리 읽기 모드로 'rag_data.pkl' 파일을 엶
    rag_data = pickle.load(f)  # 파일에서 데이터를 읽어와 'rag_data' 변수에 저장

In [2]:
questions = rag_data['questions'][:10]  # 상위 10개의 질문을 'questions'에 저장
contexts = rag_data['contexts'][:10]  # 상위 10개의 문맥을 'contexts'에 저장
answers = rag_data['answers'][:10]  # 상위 10개의 정답을 'answers'에 저장
len(questions)  # 'questions' 리스트의 길이(질문 수)를 반환

10

In [3]:
from ragas.llms import LangchainLLMWrapper  # Langchain LLM을 래핑하는 클래스 import
from langchain_openai import ChatOpenAI  # OpenAI와의 연동을 위한 Langchain 기반 클래스 import
evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))  # GPT-4o 모델을 사용해 평가 LLM 인스턴스 생성

### 평가

### Baseline (w/o RAG)

In [4]:
from tqdm import tqdm  # 진행 상태 표시를 위한 tqdm 모듈 import

from utils import call_openai, get_embeddings, cosine_similarity  # 사용자 정의 모듈에서 필요한 함수 import

predictions = []  # 예측 결과를 저장할 리스트 초기화
for i in tqdm(range(len(questions))):  # 질문 수 만큼 반복하며 진행 상태 표시
    prompt = f"""You are an expert in Finance. Please answer to the question given below.
    
Question:
{questions[i]}
"""  # 금융 분야 전문가로 가정하여 질문에 답변하도록 하는 프롬프트 생성

    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI API 호출을 통해 예측 생성
    predictions.append(prediction)  # 예측 결과를 리스트에 추가

predictions  # 생성된 예측 리스트 출력

100%|██████████| 10/10 [00:49<00:00,  4.98s/it]


['글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:\n\n1. **경제 성장 둔화**: 많은 선진국들이 경제 성장률이 둔화되고 있습니다. 경제 성장이 둔화되면 중앙은행들은 경제를 자극하기 위해 금리를 낮추는 경향이 있습니다.\n\n2. **인구 고령화**: 인구 고령화는 저축률을 높이고 소비를 줄이는 경향이 있습니다. 이는 자본의 공급을 증가시키고, 자본에 대한 수요를 감소시켜 금리를 낮추는 요인으로 작용합니다.\n\n3. **기술 발전**: 기술 발전은 생산성을 높이고, 비용을 절감하며, 인플레이션을 억제하는 경향이 있습니다. 이는 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.\n\n4. **글로벌화**: 글로벌화는 자본의 이동을 자유롭게 하고, 전 세계적으로 자본 공급을 증가시킵니다. 이는 금리를 낮추는 요인으로 작용합니다.\n\n5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 많은 중앙은행들이 양적 완화(QE)와 같은 비전통적인 통화 정책을 도입하여 금리를 낮추고, 경제를 자극하려고 했습니다. 이러한 정책들은 장기적으로 저금리 환경을 조성하는 데 기여했습니다.\n\n6. **안전 자산에 대한 수요 증가**: 경제 불확실성과 금융 시장의 변동성 증가로 인해 투자자들은 안전 자산(예: 국채)에 대한 수요가 증가했습니다. 이는 국채 금리를 낮추는 요인으로 작용합니다.\n\n이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되고 있습니다.',
 '고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높은 위험을 감수하는 투자 상품이나 자산에 집중하는 현상을 말합니다. 이러한 현상은 여러 가지 요인에 의해 발생할 수 있습니다.\n\n1. **저금리 환경**: 중앙은행의 저금리 정책으로 인해 안전한 자산(예: 국채, 예금)의 수익률이 낮아지면, 투자자들은 더 높은 수익을 추구하기 위해 고위험 자산으로 눈을 돌리게 됩니다.\n

In [5]:
from ragas.metrics import FactualCorrectness  # FactualCorrectness 메트릭 import
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset  # 평가 데이터셋 스키마와 단일 회차 샘플 import
from ragas import evaluate  # 평가 함수 import

samples = []  # 평가 샘플을 저장할 리스트 초기화
for question, ai_response, ground_truth in zip(questions, predictions, answers):  # 질문, 예측, 정답을 묶어서 반복
    sample = SingleTurnSample(
        user_input=question,  # 사용자 질문 설정
        response=ai_response,  # AI 응답 설정
        reference=ground_truth  # 참조 정답 설정
    )
    samples.append(sample)  # 평가 샘플 리스트에 추가

dataset = EvaluationDataset(samples)  # 평가 데이터셋 생성

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)  # LLM 평가를 위한 FactualCorrectness 메트릭 설정

results = evaluate(dataset, metrics=[factual_correctness_metric])  # 데이터셋과 메트릭을 사용해 평가 실행

results_df = results.to_pandas()  # 평가 결과를 DataFrame으로 변환
print(results_df.head())  # 평가 결과 상위 몇 개의 행 출력

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다...   
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 높은 수익을 기대하면서 상대적으로 높...   
2  디지털화의 진전은 20세기 이후 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행(Investment Banks, IB)들은 다양한...   
4  금융시스템 개혁법은 국가마다 다를 수 있습니다. 예를 들어, 미국의 경우 2010년...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.35  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.25  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.16  
3  2000년대 이후 글로벌 IB들은 

In [6]:
print(results)  # 전체 평가 결과 출력

{'factual_correctness': 0.1710}


In [7]:
print(questions[0])  # 첫 번째 질문 출력

글로벌 저금리 현상이 부각된 원인은 무엇인가요?


In [8]:
print(predictions[0])  # 첫 번째 예측 출력

글로벌 저금리 현상이 부각된 원인에는 여러 가지 요인이 복합적으로 작용하고 있습니다. 주요 원인들을 살펴보면 다음과 같습니다:

1. **경제 성장 둔화**: 많은 선진국들이 경제 성장률이 둔화되고 있습니다. 경제 성장이 둔화되면 중앙은행들은 경제를 자극하기 위해 금리를 낮추는 경향이 있습니다.

2. **인구 고령화**: 인구 고령화는 저축률을 높이고 소비를 줄이는 경향이 있습니다. 이는 자본의 공급을 증가시키고, 자본에 대한 수요를 감소시켜 금리를 낮추는 요인으로 작용합니다.

3. **기술 발전**: 기술 발전은 생산성을 높이고, 비용을 절감하며, 인플레이션을 억제하는 경향이 있습니다. 이는 중앙은행들이 금리를 낮게 유지할 수 있는 환경을 조성합니다.

4. **글로벌화**: 글로벌화는 자본의 이동을 자유롭게 하고, 전 세계적으로 자본 공급을 증가시킵니다. 이는 금리를 낮추는 요인으로 작용합니다.

5. **중앙은행의 통화 정책**: 2008년 금융위기 이후 많은 중앙은행들이 양적 완화(QE)와 같은 비전통적인 통화 정책을 도입하여 금리를 낮추고, 경제를 자극하려고 했습니다. 이러한 정책들은 장기적으로 저금리 환경을 조성하는 데 기여했습니다.

6. **안전 자산에 대한 수요 증가**: 경제 불확실성과 금융 시장의 변동성 증가로 인해 투자자들은 안전 자산(예: 국채)에 대한 수요가 증가했습니다. 이는 국채 금리를 낮추는 요인으로 작용합니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되고 있습니다.


In [9]:
import numpy as np  # NumPy 모듈 import
from tqdm import tqdm  # 진행 상태 표시를 위한 tqdm 모듈 import

from utils import call_openai, get_embeddings, cosine_similarity  # 사용자 정의 모듈에서 필요한 함수 import

def retrieve_context(question, contexts):  # 질문에 가장 유사한 문맥을 찾기 위한 함수 정의
    question_embedding = get_embeddings([question], model='text-embedding-3-large')[0]  # 질문 임베딩 생성
    context_embeddings = get_embeddings(contexts, model='text-embedding-3-large')  # 문맥 임베딩 생성

    similarities = [cosine_similarity(question_embedding, context_embedding) for context_embedding in context_embeddings]  # 질문과 각 문맥 간의 유사도 계산

    most_relevant_index = np.argmax(similarities)  # 가장 높은 유사도를 가지는 문맥의 인덱스 찾기
    return contexts[most_relevant_index]  # 가장 관련성이 높은 문맥 반환

predictions = []  # 예측 결과를 저장할 리스트 초기화
for i in tqdm(range(len(questions))):  # 질문 수 만큼 반복하며 진행 상태 표시
    context = retrieve_context(questions[i], contexts[i])  # 질문에 가장 유사한 문맥 찾기
    prompt = f"""You are an expert in Finance. Please answer to the question given below. Use information given in Context appropriately.

Context:
{context}

Question:
{questions[i]}
"""  # 문맥과 질문을 기반으로 답변을 생성하는 프롬프트 생성
    prediction = call_openai(prompt, model='gpt-4o-2024-05-13')  # OpenAI API 호출을 통해 예측 생성
    predictions.append(prediction)  # 예측 결과를 리스트에 추가

100%|██████████| 10/10 [00:45<00:00,  4.51s/it]


In [10]:
from ragas.metrics import FactualCorrectness  # FactualCorrectness 메트릭 import
from ragas.dataset_schema import SingleTurnSample, EvaluationDataset  # 평가 데이터셋 스키마와 단일 회차 샘플 import
from ragas import evaluate  # 평가 함수 import

samples = []  # 평가 샘플을 저장할 리스트 초기화
for question, ai_response, ground_truth in zip(questions, predictions, answers):  # 질문, 예측, 정답을 묶어서 반복
    sample = SingleTurnSample(
        user_input=question,  # 사용자 질문 설정
        response=ai_response,  # AI 응답 설정
        reference=ground_truth  # 참조 정답 설정
    )
    samples.append(sample)  # 평가 샘플 리스트에 추가

dataset = EvaluationDataset(samples)  # 평가 데이터셋 생성

factual_correctness_metric = FactualCorrectness(llm=evaluator_llm)  # LLM 평가를 위한 FactualCorrectness 메트릭 설정

results = evaluate(dataset, metrics=[factual_correctness_metric])  # 데이터셋과 메트릭을 사용해 평가 실행

results_df = results.to_pandas()  # 평가 결과를 DataFrame으로 변환
print(results_df.head())  # 평가 결과 상위 몇 개의 행 출력

Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]

                                          user_input  \
0                         글로벌 저금리 현상이 부각된 원인은 무엇인가요?   
1                     고수익-고위험 부문으로의 쏠림현상에 대해 설명해주세요.   
2  20세기 이후 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 어떤 영향을 미쳤...   
3                  2000년대 이후 글로벌 IB들이 어떤 전략을 추진하였나요?   
4                              금융시스템 개혁법은 언제 제정되었나요?   

                                            response  \
0  글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 ...   
1  고수익-고위험 부문으로의 쏠림현상은 저금리 기조 하에서 투자자들이 원하는 수익률을 ...   
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 여러 가지 중요한...   
3  2000년대 이후 글로벌 투자은행들(IB)은 여러 전략을 추진하였습니다. 주요 전략...   
4  금융시스템 개혁법에 대한 정보는 주어진 문맥에서 명확하게 언급되지 않았습니다. 그러...   

                                           reference  factual_correctness  
0  글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융...                 0.40  
1  고수익-고위험 부문으로의 쏠림현상은 투자자들이 더 높은 수익률을 추구하기 위해 위험...                 0.57  
2  20세기 이후 디지털화의 진전은 금융투자업의 사업모형과 산업구조에 큰 영향을 미쳤습...                 0.44  
3  2000년대 이후 글로벌 IB들은 

In [11]:
print(results)  # 전체 평가 결과 출력

{'factual_correctness': 0.5250}


In [13]:
print(predictions[0])  # 첫 번째 예측 출력

글로벌 저금리 현상이 부각된 원인은 여러 가지가 있습니다. 주요 원인으로는 다음과 같은 요소들이 있습니다:

1. **완화적 통화정책**: 2008년 글로벌 금융위기 이후 미국, 유럽, 일본 등 주요국들이 경제를 회복시키기 위해 완화적 통화정책을 시행하였습니다. 이러한 정책은 금리를 낮추고 유동성을 공급하여 경제를 부양하려는 목적이 있었습니다.

2. **경제구조적 요인**: 글로벌 금융위기 이전부터 이어진 경제구조적 요인들도 저금리 현상에 기여하였습니다. 예를 들어, 선진국의 성장세 둔화와 신흥국의 성장으로 인해 글로벌 무역 불균형이 심화되고, 저축과잉현상이 발생하였습니다.

3. **코로나 팬데믹**: 최근 코로나 사태로 인해 경제가 큰 타격을 받았고, 이를 극복하기 위해 각국 중앙은행들이 다시 한 번 완화적 통화정책을 강화하였습니다. 이로 인해 저금리 기조가 더욱 부각되었습니다.

이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 지속되고 있습니다.


In [12]:
print(answers[0])  # 첫 번째 정답 출력

글로벌 저금리 현상이 부각된 원인은 여러 가지입니다. 첫째, 2008년 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의 완화적 통화정책이 있습니다. 둘째, 선진국의 성장세가 둔화되고, 신흥국이 성장하면서 글로벌 무역 불균형이 심화되고 저축과잉현상이 발생했습니다. 셋째, 기술발전에 따른 자본재 가격 하락과 신기술 혁신에 따른 수익률이 낮아져 경제 전체적인 유효투자가 감소했습니다. 넷째, 세계적인 인구증가율 감소 추세는 유효수요 및 자본의 한계생산성과 투자이익 및 투자증가율을 감소시켰습니다. 따라서 이러한 요인들이 복합적으로 작용하여 글로벌 저금리 현상이 부각되었습니다.
